In [15]:
import tensorflow as tf
import keras 
from keras.applications import DenseNet201
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, NumpyArrayIterator
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.metrics import confusion_matrix
from keras import models, layers, optimizers
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline

In [5]:
conv_base = DenseNet201(weights='imagenet', include_top = False, input_shape=(300,300,3))

74842112/74836368 [==============================] - 6s 0us/step


In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

In [8]:
X = np.load("gdrive/My Drive/pcb/xtrain.npy")
y = np.load("gdrive/My Drive/pcb/ytrain.npy")

In [ ]:
conv_base.predict(X[0].reshape(1,300,300,3)).shape

(1, 9, 9, 1920)

In [ ]:
# Training set has 472 samples

Training with k-fold Validation

In [ ]:
X.shape, (y == 0).sum(), (y == 1).sum()

((472, 300, 300, 3), 236, 236)

In [9]:
datagen = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)

datagen.fit(X)

In [10]:
nos = X.shape[0]
bs = int(nos/8)

numit = NumpyArrayIterator(
    X, y, datagen, batch_size=bs, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)

samples = np.zeros(shape=(nos,9,9,1920))
labels = np.zeros(shape=(nos))
i = 0
for samples_batch,labels_batch in numit:
    f = conv_base.predict(samples_batch)
    print(f.shape)
    samples[i*bs:(i+1)*bs] = f
    labels[i*bs:(i+1)*bs] = labels_batch
    i+=1
    if (i*bs)>=nos:
        break

(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)
(59, 9, 9, 1920)


In [11]:
samples = samples.reshape(nos,9*9*1920)

In [12]:
samples.shape, labels.shape

((472, 155520), (472,))

In [13]:
# To save memory
del X
del y

In [16]:
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=8)

accuracies = []
f = 1

for train_index, test_index in skf.split(samples, labels):
        X_train = samples[train_index]
        X_test = samples[test_index]
        y_train = labels[train_index]
        y_test = labels[test_index]
      
        
        print('Fold {}'.format(f))
        clf = make_pipeline(SVC(gamma='auto'))
        clf.fit(X_train, y_train)
        
        test_predclass = clf.predict(X_test)
        
        accuracies.append([confusion_matrix(y_test,test_predclass)[0][0]/(y_test == 0.0).sum(), confusion_matrix(y_test,test_predclass)[1][1]/(y_test == 1.0).sum()])
        f+=1


Fold 1
Fold 2
Fold 3
Fold 4
Fold 5
Fold 6
Fold 7
Fold 8


In [17]:
zero_acc = 0
one_acc = 0

for i in accuracies:
  zero_acc+= i[0]
  one_acc+=i[1]

zero_acc/=8
one_acc/=8

print(zero_acc, one_acc)

0.8443965517241381 0.7886494252873564


Training on whole Training Dataset

In [18]:
xtest = np.load('gdrive/My Drive/pcb/xtest.npy')
ytest = np.load('gdrive/My Drive/pcb/ytest.npy')


In [19]:
datagentest = ImageDataGenerator(
    featurewise_center = True,
    featurewise_std_normalization = True
)
datagentest.fit(xtest)

In [20]:
testnos = ytest.shape[0]

numittest = NumpyArrayIterator(
    xtest, ytest, datagentest, batch_size=testnos, shuffle=False, sample_weight=None,
    seed=None, data_format=None, save_to_dir=None, save_prefix='',
    save_format='png', subset=None, dtype=None
)


test_samples = np.zeros(shape=(testnos,9,9,1920))
test_labels = np.zeros(shape=(testnos))
for samples_batch,labels_batch in numittest:
    g = conv_base.predict(samples_batch)
    print(g.shape)
    test_samples[::] = g
    test_labels[:] = labels_batch
    break

(552, 9, 9, 1920)


In [21]:
test_samples = test_samples.reshape(testnos,9*9*1920)

In [23]:
clftest = make_pipeline(SVC(gamma='auto'))
clftest.fit(samples, labels)
        
test_predclass = clftest.predict(test_samples)

In [24]:
confusion_matrix(test_labels,test_predclass)[0][0]/(ytest == 0.0).sum(), confusion_matrix(test_labels,test_predclass)[1][1]/(ytest == 1.0).sum()

(0.7966101694915254, 0.768762677484787)

In [ ]:
# Saving model

# model_json = model.to_json()
# with open("gdrive/My Drive/models/VGG19_12.json", "w") as json_file:
#     json_file.write(model_json)

In [ ]:
# model.save_weights("gdrive/My Drive/models/VGG19_12.h5")
# print("Saved model to disk")

In [ ]:
# Saving and loading models
# https://machinelearningmastery.com/save-load-keras-deep-learning-models/